In [86]:
function  prolongation1(n)
    r = ceil(n/2) 
    println(r)
    R = zeros(r,n)
    i = 1
    j = 1
    while i <= r 
        R[i,j] = 1
        i = i + 1
        j = j + 2
    end
    return R
end

prolongation1 (generic function with 1 method)

In [87]:
prolongation1(14)

7.0


7×14 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0

In [95]:
function  prolongation2(n)
    r = ceil(n/2)
    R = zeros(r,n)
    i = 1
    j = 1
    while i <= r 
        R[i,j] = 1
        if(j+1 <= n)
            R[i,j+1] = 2
        end
        if(j+2 <= n)
            R[i,j+2] = 1
        end
        i = i + 1
        j = j + 2
    end
    return 1/4*R
end

prolongation2 (generic function with 1 method)

In [97]:
prolongation2(15)

8×15 Array{Float64,2}:
 0.25  0.5  0.25  0.0  0.0   0.0  0.0   …  0.0  0.0   0.0  0.0   0.0  0.0 
 0.0   0.0  0.25  0.5  0.25  0.0  0.0      0.0  0.0   0.0  0.0   0.0  0.0 
 0.0   0.0  0.0   0.0  0.25  0.5  0.25     0.0  0.0   0.0  0.0   0.0  0.0 
 0.0   0.0  0.0   0.0  0.0   0.0  0.25     0.0  0.0   0.0  0.0   0.0  0.0 
 0.0   0.0  0.0   0.0  0.0   0.0  0.0      0.5  0.25  0.0  0.0   0.0  0.0 
 0.0   0.0  0.0   0.0  0.0   0.0  0.0   …  0.0  0.25  0.5  0.25  0.0  0.0 
 0.0   0.0  0.0   0.0  0.0   0.0  0.0      0.0  0.0   0.0  0.25  0.5  0.25
 0.0   0.0  0.0   0.0  0.0   0.0  0.0      0.0  0.0   0.0  0.0   0.0  0.25

In [60]:
# Solves the System Ax=b with de Deflation-Operator R
function solve(A, b, R)
    # Compute A = M - N
    M = Diagonal(A)
    M1 = M^(-1)
    N = M - A
    x = ones(size(A,1))
    
    while(norm(b-A*x)>0.1)
        x = (M1 * N * x) + (M1 * b)
    end
    x
end

solve (generic function with 1 method)

In [61]:
A = [4. 12. -16.; 12. 37. -43.; -16. -43. 98.]
R = [1. 0. 0.; 0. 0. 1.; 0. 0. 0.]
solve(A, [1.;2.;3.], 1.)

3-element Array{Float64,1}:
  NaN
 -Inf
  Inf